# R bioconductor 실습

- author: "Kwon DoHyung"
- toc: true 
- comments: true
- categories: [R, ngs, sequencing, experiment]
- image: images/2020-11-17-bioconductor/Untitled27.png
- permalink: /r-bioconductor/

# 1. bioconductor

bioconductor는 Bioinformatics 영역의 유전자 데이터 분석에 필요한, 유전체 데이터의 분석 및 이해를 돕기 위한 R 패키지다. 위키피디아에 따르면 다음과 같이 정의된다.

Bioconductor is a free, open source and open development software project for the analysis and comprehension of genomic data generated by wet lab experiments in molecular biology.(분자 생물학 실험에 의해 나온 genomic data에 대한 분석용 소프트웨어. 무료이며 오픈소스이다.)


![](../images/2020-11-17-bioconductor/Untitled.png)

가장 많은 비율로 Transcriptome에 대한 RNA sequencing이 활발하게 이루어지고 있고, Single cell RNA Sequencing이 그 다음을 차지하고 있다.

## 1-1. bioconductor 패키지 설치

패키지의 설치는 bioconductor 사이트([http://bioconductor.org](http://bioconductor.org/))에 접근 후, Install Bioconductor를 클릭하여 진행한다. R studio의 console창에서 다음을 입력한다.

```r
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install()
```

```r
BiocManager::install(c("GenomicFeatures", "AnnotationDbi"))
```

# 2. ExpressionSet

bioconductor에는 유전체 데이터 분석에 용이하게 구조화된 몇 가지 데이터 형태들이 있다. 

![](../images/2020-11-17-bioconductor/Untitled1.png)

ExpressionSet은 유전자 발현값(FPKM, TPM)등을 matrix형태로 관리할 수 있도록 표준화된 데이터 형태이다. ExpressionSet은 micro array에서부터 표준화되어 사용 되어 왔고, 여러 분석 패키지에서도 ExpressionSet의 형태나 확장된 형태로 인자를 활용하는 경우가 많다. 

ExpressionSet은 bioconductor 아래에 biobase라는 패키지를 설치하면 사용 가능하다. r biobase라고 검색하여 나오는 biobase 패키지에 대한 설명에 대한 사이트로 접근할 수 있다. 

![](../images/2020-11-17-bioconductor/Untitled2.png)

패키지에 대한 설명과 정보, 논문 등이 나와 있다.

![](../images/2020-11-17-bioconductor/Untitled3.png)

설치 방법도 나와 있다.

![](../images/2020-11-17-bioconductor/Untitled4.png)

개발자들이 작성한 문서다. 패키지 처음 사용 시, 사용법을 알기 위해 해당 페이지 등으로부터 도움을 받을 수 있다. biobase는 네 가지의 사용 설명서가 있다. 첫 번째 PDF에 ExpressionSet에 대한 설명이 있음을 알 수 있다.

## 2-1. ExpressionSet 데이터 구조

ExpressionSet은 줄여서 eSet이라고도 부른다. 한마디로 high-throughput espression level assay를 포함하고 묘사할 수 있는 데이터 형태라고 할 수 있다. eSet에는 기본적으로 다음의 세 가지 정보를 갖고 있다.

### 2-1-1. assayData

assayData는 각 샘플에서의 유전자의 실제 발현값을 가지고 있다.

- 데이터 타입: Matrix
- 행: 유전자(feature)
- 열: 샘플
- 각 값: 유전자 발현 값

![](../images/2020-11-17-bioconductor/Untitled5.png)

![](../images/2020-11-17-bioconductor/Untitled6.png)

### 2-1-2. phenoData

assay 데이터의 열에 해당하며, 각 Samples이 가지고 있는 값이다. 

- 데이터 타입: AnnotatedDataFrame
    - AnnotatedDataFrame은 기존의 R data.frame의 확장된 형태다.
- 암 환자를 대상으로 한 데이터라면, age, tumor stage 등의 임상자료들이 phenoData에 들어간다.
- 만약 실험 데이터일 경우, 실험 condition, treatment 정보, 실험적인 조건 정보들이 포함된다.

![](../images/2020-11-17-bioconductor/Untitled7.png)

### 2-1-3. featureData

assay 데이터의 Feature에 해당하는 정보가 들어간다.

- 데이터 타입: AnnotatedDataFrame
- feature가 유전자라면 chromosome, start, end, 유전자의 위치 정보, strand, 유전자 이름 등 분석에 필요한 정보들이 들어간다.

![](../images/2020-11-17-bioconductor/Untitled8.png)

이 세 가지 데이터(assayData, featureData, phenoData)가 하나의 ExpressionSet을 이룬다. 주의할 점은, Gene과 Sample의 order를 잘 맞춰줘야 한다는 점이다. 제대로 된 expressionSet을 구성하기 위해서는 assayData에서의 Gene의 order와 featureData의 gene의 order가 일치해야 하고, assayData에서의 Sample의 order와 phenoData의 Sample의 order가 일치해야 한다.

## 2-2. ExpressionSet()

```r
ExpressionSet(assayData = assay,
							phenoData = AnnotatedDataFrame(phenoData),
							featureData = AnnotatedDataFrame(featureData))
```

`ExpressionSet()` 함수를 이용하여 ExpressionSet을 만들 수 있다. `assay`는 matrix타입, `phenoData`와 `featureData`는 data.frame타입이어야 한다는 점에 유의한다. 주의할 점은, 앞에서 언급했듯이 assay의 행 이름과 feature data의 행 이름의 순서가 일치해야 한다는 점, assay의 열 이름과 pheno data의 행 이름의 순서가 일치해야 한다는 점이다.

## 2-3. ExpressionSet 정보 추출 함수들

### 2-3-1. exprs(), fData(), pData()

`exprs()`는 assay data를 추출하고, `fData()`는 feature data를 추출, `pData()`는 pheno data를 추출하는 함수다.

### 2-3-2. sampleNames(), featureNames()

ExpressionSet에서 sample의 이름과 feature의 이름을 추출하는 함수들이다.

### 2-3-3. varLabels(), fvarLabels()

data frame 형태의 feature data나 pheno data의 열의 이름을 확인할 때 사용하는 함수들이다. 

## 2-4. R 실습: ExpressionSet 구조 파악

이 노트에서는 유전체 데이터 분석에서 교과적으로 많이 사용되는 데이터인 ALL(Acute Lymphocytic Leukemia) 데이터를 이용하여 실습한다. 이 데이터는 128명의 백혈병 환자를 대상으로 B cell과 T cell에 대한 micro array 데이터다.

- BiocManager를 이용하여 ExpressionSet을 사용하기 위해 Biobase 패키지와 ALL 데이터 패키지를 설치한다.

```r
BiocManager::install('Biobase')
BiocManager::install('ALL')
```

- Biobase와 ALL 패키지를 불러온다.

```r
library(Biobase)
library(ALL)
```

- `data()` 함수로 ALL 데이터를 로드한다.

```r
data(ALL)
```

- class를 확인한다. Biobase 패키지 내의 ExpressionSet임을 확인할 수 있다.

```r
class(ALL)
```

![](../images/2020-11-17-bioconductor/Untitled9.png)

`class()` 함수의 결과에 의해, ExpressionSet임을 알 수 있다.

- ALL 데이터를 탐색하기 위해 변수 이름만 실행한다. 변수 이름만 실행하면, 데이터 내의 자료들이 출력되는 것이 아닌 데이터의 구성에 대한 대략적인 개요 정보들이 나온다.

```r
ALL
```

![](../images/2020-11-17-bioconductor/Untitled10.png)

- 세 가지 데이터 구조인 assayData, phenoData, featureData가 보인다.
- assayData 내에는 12,625개의 feature와 128개의 sample이 있는 것을 확인할 수 있다. (Q. feature가 none인데, assayData에 12,625개가 있다는 것은 무슨 소리일까? 실제 feature data에는 아무 data도 없다는 소리다. assayData의 행에 해당하는 각 probe에 대한 데이터를 직접 구해서 feature data를 구성해야 하는 상황이라고 이해하면 된다.)
- 세 가지 데이터 구조 외에 protocolData, experimentData, Annotation 등을 부가적으로 넣어줄 수 있다.
- phenoData
    - sampleNames는 sample data의 이름으로서 01005, 01010, LAL4와 같은 식으로 저장되어 있고, assayData에 나온 128개의 샘플이라는 것과 동일하게 128개의 샘플이 있음을 알 수 있다.
    - varLabels: phenoData내에 들어있는 열의 이름이다.
- 현재 ALL 데이터에는 none으로서 featureData가 들어와 있지는 않다.
- experimentData의 pubMedIds: 해당 데이터가 citation된 정보
- annotation: 현재 데이터에는 유전자 정보 대신 annotation 정보가 들어가 있다. 해당 데이터는 micro array데이터이기 때문에 해당 데이터를 만들 때 사용한 플랫폼 정보가 hgu95av2라는 의미다. 해당 플랫폼에서 만들어진 데이터라는 의미다. 해당 플랫폼에 맞는 probe 정보들을 다운로드 받아서 feature 데이터로 추가해주면 분석에 활용할 수 있다.

### 2-4-1. assayData 추출: exprs()

```r
assay <- exprs(ALL)
assay # 또는 head(assay)
```

![](../images/2020-11-17-bioconductor/Untitled11.png)

각 열은 micro array에 해당하는 플랫폼의 probe id들로 구성되어 있다.

- 차원 확인하기

```r
dim(assay)
```

[1] 12625 128

#### 2-4-1-1. sampleNames(), featureNames()

eSet에 들어있는 샘플의 이름들과 eSet에 들어있는 feature의 이름들을 출력한다. 특히, 해당 feature들은 해당하는 플랫폼의 probe id들이고, 현재는 관련 데이터가 없어서 직접 구하여 추가해야 한다. 

```r
sampleNames(ALL)
```

![](../images/2020-11-17-bioconductor/Untitled12.png)

```r
featureNames(ALL)
```

![](../images/2020-11-17-bioconductor/Untitled13.png)

![](../images/2020-11-17-bioconductor/Untitled14.png)

즉, assayData 자체에는 12,625개의 행 데이터로 probe id에 대한 정보를 담고 있지만, feature data에는 실제 probe별로 유전자 정보나 위치 정보 등을 담고 있는 정보가 없다고 이해하면 된다.

### 2-4-2. phenoData 추출: pData(ALL)

```r
phenoData <- pData(ALL)
phenoData
```

![](../images/2020-11-17-bioconductor/Untitled15.png)

![](../images/2020-11-17-bioconductor/Untitled16.png)

행에 sample의 raw name으로 지정된 이름이 보인다. 또한, assay data의 column의 순서와 pheno data의 행의 순서가 동일함을 알 수 있다. 열 영역에는 21개의 열의 이름이 보인다. 즉, 환자의 임상 정보들이 phenoData에 등록되어 있는 것을 확인할 수 있다.

#### 2-4-2-1. varMetadata()

pheno data의 각 column에 대한 설명을 볼 수 있다.

```r
varMetadata(ALL)
```

![](../images/2020-11-17-bioconductor/Untitled17.png)

### 2-4-3. featureData 추출: fData()

```r
featureData <- fData(ALL)
featureData
```

data frame with 0 columns and 12625 rows

아까 살펴 보았듯이, ALL 데이터의 featureData는 none이다. 따라서 ALL 데이터는 현재 array data과 pheno data로만 이루어져 있는 ExpressionSet 데이터라고 볼 수 있다. 

#### 2-4-3-1. annotation()

```r
annotation(ALL)
```

[1] "hgu95av2"

플랫폼 이름이 나온다. 해당 플랫폼 이름을 구글에 검색하여, public database를 통해 probe 정보를 feature로 추가하여, probe별 유전자 정보나 위치 정보 등을 확인할 수 있다.

### 2-4-4. 기타 정보 추출

#### 2-4-4-1. experimentData()

```r
experimentData(ALL)
```

![](../images/2020-11-17-bioconductor/Untitled18.png)

해당 데이터를 만든 실험자의 이름과 센터, contact 정보 등이 들어있다.

## 2-5. R 실습: RNA Sequencing data(유전자 발현량 계산된 값)를 통해 ExpressionSet 만들기

![](../images/2020-11-17-bioconductor/Untitled19.png)

- 실습에 사용할 데이터는 논문 "*EPHB6 mutation induces cell adhesion-mediated paclitaxel resistance via EPHA2 and CDH11 expression*,"([https://www.nature.com/articles/s12276-019-0261-z](https://www.nature.com/articles/s12276-019-0261-z))에 나온 데이터다.
    - 총 4 개의 RNA-seq 샘플로 이루어져 있다.
    - 간암에 관련 된 Huh7 cell line 중 EPHB6 유전자의 Mutatant Type(MUT)과 Wild Type(WT)에 대해 두 번씩 과발현(replicate)시킨 데이터다. 따라서 두 그룹 간에 어떤 유전자 관련 프로파일에 변화가 있는지 비교해보고자 한다.
    - 전처리 과정을 거친 데이터다.
- 이 노트의 실습은 `/Users/dohk/Dropbox/workspace/__bioinformatics/RNA-seq-ExpressionSet` 디렉토리에서 진행되며, R Studio 상에서 진행된다. 순서는 다음과 같이 진행된다.
    - 각 샘플에 대해 전처리를 수행한 후, cufflinks를 이용하여 유전자 발현량을 계산한다(RSEM과 같은 일을 하는 tool임). 각 샘플의 유전자 발현량을 cufflinks를 이용하여 계산한 결과(gene.fpkm_tracking)를 R로 불러온다.
        - `/Users/dohk/Dropbox/workspace/__bioinformatics/RNA-seq-ExpressionSet/result/cufflinks` 디렉토리에 cufflinks를 다음 하위 폴더들이 있다.

![](../images/2020-11-17-bioconductor/Untitled20.png)

각 폴더들에 대한 트리구조를 그려보면 다음과 같다. 실제 디렉토리에는 빨간색 박스 친 파일만 존재한다. cufflinks를 돌리면 저 파일들이 모두 생성된다.

![](../images/2020-11-17-bioconductor/Untitled21.png)

파일 내부의 내용은 다음과 같다.

![](../images/2020-11-17-bioconductor/Untitled22.png)

- 그 후, 각 샘플의 cufflinks 데이터로부터 FPKM column을 추출해 assay data를 생성한다.
- gene annotation data(gencode.v22.df.rda)를 불러와 featureData를 생성한다.
- phenotypic group 정보를 이용해 phenoData를 생성한다.
- Assay, featureData, phenoData로부터 ExpressionSet을 생성한다.
- 이를 통해 데이터의 분포를 간단히 확인해 볼 수 있다.

### 2-5-1. 실습 준비

#### 2-5-1-1. 프로젝트 생성

![](../images/2020-11-17-bioconductor/Untitled23.png)

![](../images/2020-11-17-bioconductor/Untitled24.png)

![](../images/2020-11-17-bioconductor/Untitled25.png)

프로젝트 생성 후, 파일의 이름을 지정한다.

![](../images/2020-11-17-bioconductor/Untitled26.png)

#### 2-5-1-2. 로드맵

지금부터 할 실습의 큰 그림은 다음과 같다. 각 파일로부터 assayData, phenoData, featureData를 생성하여 최종적으로 ExpressionSet으로 만드는 것이 목표다. 각 데이터이 타입에 주의해야 한다.

![](../images/2020-11-17-bioconductor/Untitled27.png)

### 2-5-2. 데이터 준비하기

필요한 라이브러리인 Biobase를 불러와 준비한다.

```r
library("Biobase")
```

cufflinks를 통해 나온 결과를 변수에 저장한다. `" "` 내에서 tab 키를 누르면 경로 상 이동이 가능하다. 방법 2에서는 디렉토리 네임을 지정해 준 다음, 해당 디렉토리 내의 서브 디렉토리들을 recursive하게 방문하면서 pattern에 맞는 파일들을 찾아 `cuff.fns`라는 변수에 저장한 후, 해당 변수를 이용하여 delim() 함수를 일괄 적용하는 lapply() 함수를 이용하는 좀 더 자동화된 코드다.

```r
# 방법 1
cuff.mut1 <- read.delim("result/cufflinks/EPHB6_MUT1/genes.fpkm_tracking")
cuff.mut2 <- read.delim("result/cufflinks/EPHB6_MUT2/genes.fpkm_tracking")
cuff.wt1 <- read.delim("result/cufflinks/EPHB6_WT1/genes.fpkm_tracking")
cuff.wt2 <- read.delim("result/cufflinks/EPHB6_WT2/genes.fpkm_tracking")

# 방법 2
cuff.dir <- "result/cufflinks/"
cuff.fns <- dir(cuff.dir, pattern = "genes.fpkm_tracking", recursive = T, full.names = T)
cuff.outs <- lapply(cuff.fns, read.delim) # results are stored to 'list' type 
names(cuff.outs)=basename(dirname(cuff.fns))
```

저장한 변수의 클래스를 확인해보면 숫자와 문자가 섞여 있기 때문에 data.frame임을 확인할 수 있다.

```r
# 방법 1
class(cuff.mut1)
class(cuff.mut2)
class(cuff.wt1)
class(cuff.wt2)

# 방법 2
sapply(cuff.outs, class)
```

- 방법 1의 결과: [1] "data.frame"
- 방법 2의 결과:

![](../images/2020-11-17-bioconductor/Untitled28.png)

실제 데이터를 살펴보기 위해 `head`를 이용하여 출력한다.

```r
# 방법 1
head(cuff.mut1)

# 방법 2
lapply(cuff.outs, head)
```

![](../images/2020-11-17-bioconductor/Untitled29.png)

총 13개의 column이 보인다. R studio의 오른쪽 위, Environment를 보면 데이터들이 모두 60,556개의 row와 13개의 column으로 이루어진 것을 알 수 있다. tracking_id와 gene_id가 같다. 특히, gene_id는 gencode에서 사용하는 ensemble id이기 때문에, gene_id만으로는 어떤 유전자인지 식별할 수 없다. 따라서 gene_short_name으로 제공된다. loucs에는 각 gene의 chromosome상의 어디에 존재하는지에 대한 정보다. FPKM값은 우리가 다루어야 할 값이다. 편차가 적을수록 좋은 QC값을 갖는다고 할 수 있다.

```r
# 방법 1
nrow(cuff.mut1)
nrow(cuff.mut2)
nrow(cuff.wt1)
nrow(cuff.wt2)

# 방법 2
sapply(cuff.outs, nrow)
```

- 방법 1의 결과:
    - [1] 60556
    - [1] 60556
    - [1] 60556
    - [1] 60556
- 방법 2의 결과:

![](../images/2020-11-17-bioconductor/Untitled30.png)

행의 개수가 60,556개인 것을 알 수 있다. 이제, geneID의 order가 같은지를 체크한다. tracking_id가 같은지를 살핀다. 

```r
cuff.mut1(달러표시)tracking_id==cuff.mut2(달러표시)tracking_id
```

order가 같다면 전부 TRUE가 나와야 하는데, FALSE가 섞여 있다. 따라서 두 파일의 tracking_id가 다르다는 결론을 내릴 수 있다. 먼저, 서로 다른 것들의 개수를 세어 보고 난 후, 기준으로 삼을 unique한 유전자 id를 만든다.

```r
sum(!(cuff.outs(달러표시)EPHB6_MUT1(달러표시)tracking_id == cuff.outs(달러표시)EPHB6_MUT2(달러표시)tracking_id))

# 방법 1
cuff.ids <- c(cuff.mut1(달러표시)tracking_id, cuff.mut2(달러표시)tracking_id, cuff.wt1(달러표시)tracking_id, cuff.wt2(달러표시)tracking_id)
length(cuff.ids)
length(key_ids)

key_ids <- unique(cuff.ids)

# 방법 2
cuff.ids <- unlist(lapply(cuff.outs, function(a) as.character(a(달러표시)tracking_id))) # get ID of each element

length(cuff.ids) # 242224

kid <- unique(cuff.ids) # key ids, unique()
length(kid) # 60468
```

- 방법 1의 결과:
    - [1] 242224
    - [1] 60468
- 방법 2의 결과(서로 다른 것들의 개수를 알려준다.):
    - [1] 49092

두 id의 길이가 다른 것을 확인할 수 있다.

### 2-5-3. assayData 만들기

네 개의 파일을 R 상에서 읽은 후, ExpressionSet을 만들기 위해 가장 먼저 할 일은 assay data를 만드는 것이다. 네 개의 파일 각각에 있는 FPKM값을 추출하여 행과 열을 맞춰 matrix 타입의 assay data를 만드는 것이 목표다. 이 때, order를 맞추는 게 어려운 작업이 될 수 있다. 

![](../images/2020-11-17-bioconductor/Untitled31.png)

간단하게 도식화 한 그림이다. 

1. 네 파일의 gene id를 unique한 key id로 만든 후 이를 행(row)으로, 샘플을 열(column)로 하여 아무 값도 들어있지 않은(NA) 빈 matrix를 만든다. 
2. 각 파일로부터 FPKM column에 해당하는 값을 뽑아, matrix에 순차적으로 넣는다. 
    - 이렇게 하는 이유는, 아무리 동일한 tool을 이용하여 결과를 냈다 하더라도 파일마다 order에 차이가 있을 수 있기 때문이다.

#### 2-5-4-1. Empty matrix 만들기

matrix를 만드는 함수를 이용해 NA matrix를 만든다.

```r
assay.data <- matrix(NA, nrow = length(key_ids), ncol=4)
head(assay.data)
```

![](../images/2020-11-17-bioconductor/Untitled32.png)

행과 열의 이름을 지정해주기 위해 다음과 같이 한다.

```r
rownames(assay.data) <- key_ids
colnames(assay.data) <- c("EPHB6_MUT1", "EPHB6_MUT2", "EPHB6_WT1", "EPHB6_WT2")
# 또는 colnames(assay) <- names(cuff.outs)
head(assay.data)
```

![](../images/2020-11-17-bioconductor/Untitled33.png)

비어있는 각 NA값을 채우기 위해 FPKM값을 이용해 값을 채운다. `match`를 이용하며, `match(기준이 되는 벡터, 찾고자 하는 벡터)`와 같이 사용한다.

```r
match(key_ids, cuff.mut1(달러표시)tracking_id)
match(key_ids, cuff.mut2(달러표시)tracking_id)
match(key_ids, cuff.wt1(달러표시)tracking_id)
match(key_ids, cuff.wt2(달러표시)tracking_id)
```

![](../images/2020-11-17-bioconductor/Untitled34.png)

cuff.mut1의 모든 tracking_id에서 matching되는 tracking id들을 찾은 것으로 보인다. 이제, 해당 tracking_id의 FPKM값을 가져와 NA값 대신에 채워주기 위해 다음가 같이 한다. assay 데이터는 네 개의 파일이 하나로 구성된 데이터이므로, 첫 번째 열에 MUT1 데이터, 두 번째 열에 MUT2 데이터, 세 번째 열에 WT1 데이터, 네 번째 열에 WT2 데이터가 들어가도록 구성한다.

```r
assay.data[,1] <- cuff.mut1(달러표시)FPKM[match(key_ids, cuff.mut1(달러표시)tracking_id)]
assay.data[,2] <- cuff.mut2(달러표시)FPKM[match(key_ids, cuff.mut2(달러표시)tracking_id)]
assay.data[,3] <- cuff.wt1(달러표시)FPKM[match(key_ids, cuff.wt1(달러표시)tracking_id)]
assay.data[,4] <- cuff.wt2(달러표시)FPKM[match(key_ids, cuff.wt2(달러표시)tracking_id)]
```

값이 제대로 들어가 있는지 확인한다.

```r
head(assay.data)
```

또는 다음과 같이 한 번에 수행할 수도 있다.

```r
# 방법 2
for(i in 1:ncol(assay)) {
  assay[as.character(cuff.outs[[i]](달러표시)tracking_id),i] <- cuff.outs[[i]](달러표시)FPKM
}
head(assay)
```

![](../images/2020-11-17-bioconductor/Untitled35.png)

### 2-5-4. phenoData 만들기

![](../images/2020-11-17-bioconductor/Untitled36.png)

각 샘플에는 sample ID가 존재한다. sample ID에는 각 샘플에 대한 정보가 들어가는데, 여기에서는 MUT(Mutant Type)인지, WT(Wild Type)인지에 대한 정보를 이용하여 data.frame 타입의 데이터를 만든다. 이 실습에서는 phenoData를 sample id와 group 정보로 만든다.

```r
sample.id <- colnames(assay.data)
head(sample.id)

group <- c("MUT", "MUT", "WT", "WT")
head(group)

pheno.data <- data.frame(sample.id, group)
head(pheno.data)
```

- [1] "EPHB6_MUT1" "EPHB6_MUT2" "EPHB6_WT1" "EPHB6_WT2"
- [1] "MUT" "MUT" "WT" "WT"

![](../images/2020-11-17-bioconductor/Untitled37.png)

행(row)의 이름이 1,2,3,4 등으로 지정이 따로 안되어 있는 상태이므로, 이를 지정해준다. phenoData의 행의 이름은 assayData의 열의 이름이 된다.

```r
rownames(pheno.data) <- sample.id
head(pheno.data)
```

지금까지의 phenoData 만드는 과정을 한 번에 수행할 수 있는 코드는 다음과 같다.

```r
#방법 2
phenoData <- data.frame(sample.id=colnames(assay), group=gsub("EPHB6_|[0-9](달러표시)", "", colnames(assay)))
rownames(phenoData) <- phenoData(달러표시)sample.id
phenoData
```

![](../images/2020-11-17-bioconductor/Untitled38.png)

### 2-5-5. featureData 만들기

feature data는 reference로 사용한 유전자 annotation GTF 파일로부터 생성한다. GTF 파일엔 유전자 뿐만 아니라 유전자 내 exon, 유전자 isoform 등의 정보가 들어 있기 때문에 용량이 크지만, 여기서는 gene 정보만 들어있는 파일을 쓴다; `genecoe.v22.df.rda`. 이 파일은 cufflinks를 돌려 alignment를 수행할 때 사용한 GTF파일 중 gene 정보만 갖고 있는 행(row) 부분만 파싱 된 데이터다. 이 데이터를 불러오고 내용을 살펴보자.

```r
data("gencode.v22.df")
head(gencode.v22.df)
```

![](../images/2020-11-17-bioconductor/Untitled39.png)

- `seqnames`: chromosome 정보
- `start`, `end`: start position과 end postition 정보
- `width`: 길이 정보
- `strand`: strand 정보로서, RNA이므로 방향을 나타냄
- `gene_id`: assayData를 만들 때 `key_ids`로 사용했던 정보. 이 정보를 이용해 order를 matching할 예정이다.
- `gene_type`: 유전자의 유형에 대한 정보

위 정보들이 ExpressionSet의 feature가 된다. 이제 featureData를 만든다.

```r
matched.idx <- match(key_ids, gencode.v22.df(달러표시)gene_id)
feature.data <- gencode.v22.df[matched.idx,]
head(feature.data)
```

![](../images/2020-11-17-bioconductor/Untitled40.png)

```r
key_ids[1:5]
```

![](../images/2020-11-17-bioconductor/Untitled41.png)

key_ids의 1~5번째와 feature.data의 1~5번째의 order가 일치함을 확인할 수 있다. rownames를 이용하여 행의 이름을 정해준다. featureData의 행의 이름은 assayData의 행의 이름이 된다.

```r
rownames(feature.data) <- key_ids
head(feature.data)
```

![](../images/2020-11-17-bioconductor/Untitled42.png)

### 2-5-6. ExpressionSet 만들기

2-5-2 ~ 2-5-6 에서 만든 데이터를 이용하여 ExpressionSet을 만든다. 세 개의 데이터를 모두 만들었으므로 이를 통해 ExpressionSet을 만들 수 있다. 

#### 2-5-6-1. 각 데이터의 order 체크

assayData의 열의 이름은 phenoData의 행의 이름과 그 순서가 같아야 하며, 현재는 sample의 이름이다. 또한, assayData의 행의 이름은 featureData의 행의 이름과  그 순서가 같아야 한다. 따라서 element-wise 연산을 통해 각 원소마다 일치하는지를 확인한다. 모든 데이터에 대해 다루기 때문에, 조건문을 써서 다른 게 있는지 없는지 정도만 판단한다. 또는, 조건문 없이 TRUE는 0, FALSE는 1이라는 원리를 이용할 수도 있다.

```r
if(colnames(assay.data) != rownames(pheno.data)) {
  print('FALSE exist')
} else print('ALL TRUE')

if(rownames(assay.data) != rownames(feature.data)) {
  print('FALSE exist')
} else print('ALL TRUE')

sum(colnames(assay.data) != rownames(pheno.data))
sum(rownames(assay.data) != rownames(feature.data))
```

- [1] "ALL TRUE"
- [1] "ALL TRUE"
- [1] 0
- [1] 0

#### 2-5-6-2. phenoData, featureData를 AnnotatedDataFrame으로 만들기

phenoData, featureData를 AnnotatedDataFrame화 하지 않고 그대로 ExpressionSet으로 만들면 다음과 같은 오류가 나온다.

```r
ExpressionSet(assayData = assay.data,
              phenoData = pheno.data,
              featureData = feature.data)
```

![](../images/2020-11-17-bioconductor/Untitled43.png)

따라서 다음과 같이 AnnotatedDataFrame() 함수를 이용하여 데이터 타입을 바꿔준 후, ExpressionSet으로 만든다.

```r
pheno.data <- AnnotatedDataFrame(pheno.data)
feature.data <- AnnotatedDataFrame(feature.data)

eSet <- ExpressionSet(assayData = assay.data,
                      phenoData = pheno.data,
                      featureData = feature.data) 
eSet
```

![](../images/2020-11-17-bioconductor/Untitled44.png)

해당 ExpressionSet을 저장한다.

```r
save(eSet, file = 'data/eSet.rda')
```

![](../images/2020-11-17-bioconductor/Untitled45.png)

이렇게 여러 FPKM값 관련 파일들을 하나의 matrix 형태로 정리된 ExpressionSet 데이터를 만들었다. 이 데이터를 이용하여 그룹 내의 샘플 간 평균값의 차이만을 이용하여 유전자를 선별하고 heatmap으로 visualization할 수도 있다. 

## 2-6. ExpressionSet 활용하기

### 2-6-1. 유전자 별에 따라 그룹 별 발현된 유전자 개수 세기

발현된 유전자의 개수를 세기 위해서는 FPKM값이 있는지 없는지만 조사하면 된다. 네 개의 그룹에서 모두 발현되었는지 확인할 수도 있다.

```r
head(exprs(eSet)) # exprs는 ExpressionSet을 추출하는 함수다.
zero.count <- rowSums(exprs(eSet)==0) # TRUE=1, FALSE=0
head(zero.count)

filter.index_1 <-  zero.count < 4
head(filter.index_1)
```

![](../images/2020-11-17-bioconductor/Untitled46.png)

### 2-6-2. 단백질로 번역되는 유전자 찾기

2-6-1에서 모든 그룹에서 발현된 유전자와 featureData의 gene_type이 "protein_coding"에 해당할 경우를 추출하여 이를 새로운 ExpressionSet으로 만든다. 이는 단백질로 번역되는 케이스를 찾아내기 위한 과정이다.

```r
head(fData(eSet)) # fData는 featureData를 추출하는 함수다.

filter.index_2 <- fData(eSet)(달러표시)gene_type=="protein_coding"

filter.index <- filter.index_1 & filter.index_2

filtered.eset <- eSet[filter.index,] # 새로운 ExpressionSet으로 만든다.
filtered.eset
```

![](../images/2020-11-17-bioconductor/Untitled47.png)

### 2-6-3. Visualization

ExpressionSet 타입인 filtered.eset으로부터 히스토그램을 그려 분포를 살피고, boxplot을 그려 그룹 별 발현된 유전자의 대략적인 분포를 살핀다.

```r
hist(exprs(filtered.eset))
boxplot(exprs(filtered.eset))
```

참고로, boxplot에서 stats를 이용하여 boxplot에 대한 통계값을 확인할 수 있다.

```r
boxplot(rnorm(10))(달러표시)stats
```

![](../images/2020-11-17-bioconductor/Untitled80.png)

![](../images/2020-11-17-bioconductor/Untitled81.png)

5개의 값 각각은, 순서대로 1Q(최소값), 2Q, 3Q, mean 또는 median, 4Q(최대값)을 의미한다.

#### 2-6-3-1. histogram과 boxplot

![](../images/2020-11-17-bioconductor/Untitled48.png)

![](../images/2020-11-17-bioconductor/Untitled49.png)

결과로 나온 그래프를 보면, 값이 너무 한 쪽에 몰려있다. 이렇게 값이 너무 크거나 작을 경우에는 값을 로그로 보정하여 시각화하면 분포를 살피는 데 도움이 된다.

```r
log2.eset <- filtered.eset
exprs(log2.eset) <- log2(exprs(filtered.eset)+1)

hist(exprs(log2.eset))
boxplot(exprs(log2.eset))
```

![](../images/2020-11-17-bioconductor/Untitled50.png)

![](../images/2020-11-17-bioconductor/Untitled51.png)

- 각 값을 `apply()` 함수를 이용하여 중간값(median)으로 보정해준다.
- apply 계열 함수는 주어진 함수 연산을 특정 단위로 쉽게 수행할 수 있도록 지원하는 함수 군이다. 어떤 함수냐에 따라 1) 연산 대상 데이터의 종류, 2) 결과 출력 형태, 3) 연산 단위 등이 달라진다. apply 계열의 함수는 사용하기가 조금 까다롭지만, 익혀두면 for, while 등의 반복문 보다 빠른 속도와 짧은 코드로 반복 연산을 처리할 수 있다.
- 이 중, `apply()` 함수는 apply 계열 함수 중 가장 기본이 되는 함수로, 행(Row) 또는 열(Column) 단위의 연산을 쉽게 할 수 있도록 지원하는 함수다. `apply()` 함수의 Input Data는 배열, 매트릭스 등 모두 같은 변수형을 가진 값으로 이루어진 데이터 타입만 가능하다. 모두 같은 변수형을 가진 데이터프레임도 가능하다. `apply()` 함수의 실행 결과는 매트릭스나 벡터로 출력된다. 파라미터에 대한 설명은 다음과 같다.

```r
apply(Input_Data, 행 또는 열을 결정하는 숫자(행: 1, 열: 2, 연산 또는 함수)
```

```r
# Array median centering
expr <- exprs(log2.eset)
apply(expr, 2, median) 

expr <- apply(expr, 2, function(a) a-median(a, na.rm=T)) # apply()
apply(expr, 2, median) 

acen.eset <- log2.eset
exprs(acen.eset) <- expr

hist(exprs(acen.eset))
boxplot(exprs(acen.eset))
```

![](../images/2020-11-17-bioconductor/Untitled52.png)

![](../images/2020-11-17-bioconductor/Untitled53.png)

음의 값이 나오지 않도록 mad 연산을 적용한다.

```r
# select variably expressed genes
data <- exprs(acen.eset)
mad <- apply(data, 1, mad) # MAD of each gene
quantile(mad)

dim(data[mad>0.1,])
```

[1] 5730 4

![](../images/2020-11-17-bioconductor/Untitled54.png)

#### 2-6-3-2. heatmap

```r
# heatmap
# install.packages("gplots")
# install.packages("lowess")
library(gplots)
breaks=seq(-2, 2, 0.1)
col=colorRampPalette(c("navy","white", "red"))(length(breaks)-1)

centered.data <- data-rowMeans(data)
heatmap.2(centered.data[mad>0.1,], trace="none", breaks=breaks, col=col, Colv= T, Rowv= T, labRow="")
```

![](../images/2020-11-17-bioconductor/Untitled55.png)

##### 2-6-3-2-1. 두 그룹 간의 변화 계산하기

```r
# Calculate Fold changes between two groups
acen.eset(달러표시)group

mut.expr <- exprs(acen.eset)[,1:2] # equal to exprs(acen.eset)[,acen.eset(달러표시)group=='MUT']
wt.expr <- exprs(acen.eset)[,3:4] # equal to exprs(acen.eset)[,acen.eset(달러표시)group=='WT']

# fold change: means of Case (MUT) group - means of Control (WT) group
mut.mean <- rowMeans(mut.expr)
wt.mean <- rowMeans(wt.expr)

fc <- mut.mean - wt.mean

# add fc to featureData of eset
fData(acen.eset)(달러표시)fc <- fc
head(fData(acen.eset))

hist(fData(acen.eset)(달러표시)fc)
```

![](../images/2020-11-17-bioconductor/Untitled56.png)

### 2-6-4. DEG (Differential Expression Gene). 차별 유전자 발현

특별히 expression이 많이 일어나거나 적게 일어나는 gene들을 DEG라고 한다. 이 분석은 동일한 유전자의 평균 발현량이 서로 다른 조건에서 유의하게 다른지를 분석하는 방법론이다. R에는 대표적으로 limma 패키지가 있다.

```r
## DEGs by fold change cutoff
# eset order by fc
ordered.eset <- acen.eset[order(fData(acen.eset)(달러표시)fc, decreasing = T),]

# DEG condition (fold change > 0.5
deg.up.index <- fData(ordered.eset)(달러표시)fc > 0.5
deg.dn.index <- fData(ordered.eset)(달러표시)fc < -0.5

head(fData(ordered.eset)[deg.up.index,])
```

![](../images/2020-11-17-bioconductor/Untitled57.png)

```r
degID.up <- featureNames(ordered.eset)[deg.up.index]
degID.dn <- featureNames(ordered.eset)[deg.dn.index]

DEG.UP=fData(ordered.eset)[degID.up,"gene_name"]
DEG.DOWN=fData(ordered.eset)[degID.dn,"gene_name"]

cat(DEG.UP, file = "result/EPHB6_MUTvsWT_DEG_UP.txt")
cat(DEG.DOWN, file = "result/EPHB6_MUTvsWT_DEG_DOWN.txt")

data <- exprs(ordered.eset[c(degID.up, degID.dn),])
dim(data)

centered.data <- data - rowMeans(data)
hist(centered.data)
heatmap.2(centered.data, trace="none", breaks=breaks, col=col, Colv= F, Rowv= F, labRow="", cexCol = 0.7)
```

![](../images/2020-11-17-bioconductor/Untitled58.png)

![](../images/2020-11-17-bioconductor/Untitled59.png)

# 3. GenomicRanges(GRanges)

![](../images/2020-11-17-bioconductor/Untitled60.png)

GenomicRange 패키지는 유전자의 위치, 영역 정보를 다루고자 할 때 주로 사용되는 유전체 영역에 관련된 표현, 분석을 위한 패키지다. 데이터 타입은 GRange라는 데이터 타입이다. data.frame과 비슷하다고 볼 수 있다. 특정 유전자 또는 유전자에 해당 하는 위치 정보, P value 등을 담고 있다. BiocManager를 이용한 다음 명령어를 통해 설치 및 사용할 수 있다.

```r
BiocManager::install("GenomicRanges")
library(GenomicRanges)
```

## 3-1. GRanges 만들기

가장 기본이 되는 데이터인 GRanges를 만드는 방법부터 살펴보자. 주로 유전체 상의 영역을 표현하기 위한 데이터다. `GRanges()` 함수로 만들 수 있다.

```r
gr <- GRanges(seqnames = c("chromosome1", "chromosome2", "chromosome3"),
							ranges = IRanges(start = c(1, 1, 1), end = c(10, 20, 30)),
							strand = c("-", "+", "+"))	
```

`GRanges()` 함수의 첫 번째 인자인 `seqnames`파라미터 에는 chromosome에 대한 정보가 들어간다. range 파라미터에는 범위를 지정해주는 `IRanges()` 함수를 이용하여 지정해준다. 위 예시에 따르면 이 유전자는 chromosome 1 번의 1번~10번 영역에 위치해 있고, chromosome 2 번에는 1~20번 영역에, chromosome 3 번에는 1~30번 영역에 위치해 있음을 알 수 있다. RNA에 대한 정보이므로, stand가 역방향인지 정방향이지에 대한 정보를 `strand` 파라미터에 기술한다.

![](../images/2020-11-17-bioconductor/Untitled61.png)

`seqnames`, `ranges`, `strand`. 이 세 파라미터가 가장 기본적인 GRanges 데이터의 구조를 이룬다. 이 외에 추가적으로 더 많은 데이터를 추가할 수 있다. 

```r
# 방법 1
gr <- GRanges(seqnames = c("chr1", "chr2", "chr3"), 
              ranges = IRanges(start=c(1, 1, 1), end=c(10, 20, 30)), 
              strand = c("-", "+", "+"),
              score = c(1,2,3),
              name = c("a", "b", "c"))

gr

# 방법 2
gr <- GRanges(seqnames = c("chr1", "chr2", "chr3"), 
			        ranges = IRanges(start=c(1, 1, 1), 
															 end=c(10, 20, 30)), 
							strand = c("-", "+", "+"))
gr(달러표시)score = 1:3
gr(달러표시)name = letters[1:3]
gr
```

![](../images/2020-11-17-bioconductor/Untitled62.png)

- 방법 1처럼 구성할 수도 있지만, 마치 data.frame타입의 데이터에 새로운 데이터를 추가할 때처럼 달러표시를 이용하여 새로운 컬럼을 만들고 내용을 추가할 수 있다. 또한, data.frame 타입의 데이터처럼 숫자든 문자열이든 상관없이 하나의 데이터로 구성할 수 있는 모습을 볼 수 있다.

입력한 데이터를 추출할 수 있는 방법은 다음과 같이 `seqnames()`, `start()`, `end()`, `strand()`와 같이 `GRanges()` 함수를 만들 때 사용했던 파라미터를 이용하여 호출할 수 있다. `as.character()` 함수를 이용하면 character 형태로 변환이 가능하다.

```r
seqnames(gr)
```

![](../images/2020-11-17-bioconductor/Untitled63.png)

```r
start(gr)
```

[1] 1 1 1

```r
end(gr)
```

[1] 10 20 30

```r
strand(gr)
```

![](../images/2020-11-17-bioconductor/Untitled64.png)

위 결과는 역방향(-) 1개, 정방향(+) 2개임을 의미한다. 

```r
as.character(seqnames(gr))
```

[1] "chr1" "chr2" "chr3"

```r
as.character(strand(gr))
```

[1] "-" "+" "+"

```r
length(gr) # 3
```

`length()` 함수는 GRanges 데이터가 몇 개의 영역 정보를 가지고 있는지 알려준다. 각 영역이 가지고 있는 넓이 정보는 `width()` 함수를 통해 파악할 수 있다.

```r
width(gr)
```

[1] 10 20 30

```r
mcols(gr)
```

`mcols()` 함수는 GRanges 데이터에 추가로 붙여준 데이터를 data.frame의 형태로 보여준다.

![](../images/2020-11-17-bioconductor/Untitled65.png)

GRanges 데이터에 추가로 붙여준 데이터는 달러기호를 통해서도 호출이 가능하다. 단, 반드시 필요한 3개의 파라미터(`seqnames`, `IRanges`, `start`, `end`, `strand`)는 호출이 불가능하다.

```r
gr(달러표시)score
gr(달러표시)name
```

![](../images/2020-11-17-bioconductor/Untitled66.png)

## 3-2. findOverlaps() 함수

이 함수는 어떤 두 영역에 대해서 어느 영역이 오버랩되고, 오버랩되는 각각의 쌍은 어떻게 되는지를 알려 준다. 특정 영역 내에 어떤 유전자가 속해 있는지를 파악하고자 할 때 사용된다. 이 노트의 실습에서는 아까 만든 ExpressionSet인 `eSet.rda`를 이용하여 featureData의 gene 영역 정보로 GRanges 데이터를 만들고, 새로운 영역(`chr17:7582285-7773654`, `chr13:47501983-49283632`) 을 추가하여 해당 영역에 어떤 유전자가 있는지 찾아내기 위해 `findOverlaps()` 함수를 이용하여 유전자 리스트를 만들고자 한다.

```r
library(GenomicRanges)

data("eSet")
eSet
```

먼저, RGanges 데이터를 만들기 위해 GenomicRanges 패키지를 library() 함수를 이용해 불러온다. 그 다음, 아까 저장한 eSet.rda 파일을 불러오고 출력해본다. 네 개의 샘플에 해당하는 ExpressionSet을 확인할 수 있다.

![](../images/2020-11-17-bioconductor/Untitled67.png)

### 3-2-1. featuerData를 GRanges 데이터로 만들기

featureData를 살펴보면 다음과 같다.

```r
head(fData(eset))
```

각 샘플이 행의 이름으로, 샘플별 feature가 열의 이름으로 들어가 있다. 해당 데이터를 보면, `seqnames`, `start`, `end` 등의 위치 정보가 보이고 `width`와 `strand`도 보인다. 

![](../images/2020-11-17-bioconductor/Untitled68.png)

```r
fdata.ft <- GRanges(seqnames = fData(eSet)(달러표시)seqnames,
										ranges = IRanges(start = fData(eSet)(달러표시)start,
												 end = fData(eSet)(달러표시)end),
										strand = fData(eSet)(달러표시)strand)
fdata.gr
```

![](../images/2020-11-17-bioconductor/Untitled69.png)

```r
fdata.gr[1:10]
```

![](../images/2020-11-17-bioconductor/Untitled70.png)

이렇게 만들어진 GRanges 데이터는 이제부턴 data.frame처럼 편하게 새로운 데이터를 추가 또는 추출할 수 있다. 이 실습 섹션의 목적은 특정 위치에 해당하는 유전자가 무엇인지 알아내는 것이므로, gene_name도 GRanges 데이터에 추가해준다. 

```r
fdata.gr(달러표시)gene_name <- fData(eSet)(달러표시)gene_name
fdata.gr
```

![](../images/2020-11-17-bioconductor/Untitled71.png)

### 3-2-3. chromosome 상의 특정 위치 찾기

```r
#chr17:7582285-7773654
#chr13:47501983-49283632
```

위 두 chromosome에 대한 GRanges를 만든다. 만들 때 `start`와 `end`에 주의한다. `strand`는 넣지 않아도 된다.

```r
gr <- GRanges(seqnames = c("chr17", "chr13"),
              ranges = IRanges(start = c(47501983, 7582285),
                               end = c(49283632, 49283632)))
gr
```

![](../images/2020-11-17-bioconductor/Untitled72.png)

목적은 두 영역에 해당하는 유전자가 무엇인가 하는 것이다. 이 때 사용할 수 있는 함수가 `findOverlaps()` 함수다.

### 3-2-4. findOverlaps()

두 영역이 있을 때 영역끼리 겹치는 쌍을 찾을 때 쓰는 함수다. `query` 파라미터는 찾고자 하는 유전자의 영역이다. `subject` 파라미터에는 어디에서 찾을 지를 알려준다. 즉 find의 대상이 되는 GRanges 데이터가 된다.

```r
overlaps <- findOverlaps(query = gr, subject = fdata.gr)
overlaps
```

![](../images/2020-11-17-bioconductor/Untitled73.png)

첫 번째 컬럼인 queryHits는 `gr` 변수에 넣어준 두 chromosome인 `chr17`과 `chr13`에 대한 인덱스 값으로 각각 `1`과 `2`가 된다. 두 번째 컬럼인 subjectHits는 `fdata.gr` 변수의 인덱스 값이다. 첫 번째 행을 보면 chr17의 영역이 subject의 24,576 부분과 overlap이 된다는 의미다. 두 번째 행을 보면 chr17의 영역이 subject의 22,967 번째 영역과 겹친다는 의미다. findOverlaps() 함수에 의해 생성된 overlaps 변수의 값은 다음과 같이 추출할 수 있다.

```r
queryHits(overlaps)
```

![](../images/2020-11-17-bioconductor/Untitled74.png)

```r
subjectHits(overlaps)
```

![](../images/2020-11-17-bioconductor/Untitled75.png)

```r
length(queryHits(overlaps)) == length(subjectHits(overlaps))
```

- [1] TRUE

둘의 길이는 같다. 이제 해당하는 유전자가 무엇인지 알아보자. 인덱스를 통해 해당 유전자를 찾는 것이다.

```r
corresponding_gene_name <- fdata.gr(달러표시)gene_name[subjectHits(overlaps)]
length(corresponding_gene_name)
corresponding_gene_name
```

![](../images/2020-11-17-bioconductor/Untitled76.png)

chr17(인덱스 1번)과 chr13(인덱스 2번) 모두에 해당하는 gene name이 나온다. 이 중 chr17(인덱스 1번)에만 해당하는 유전자 이름을 뽑아본다. 

```r
idx.queryHits <- queryHits(overlaps)==1
idx.queryHits
```

1번 인덱스에 해당하는 것들에 TRUE로 표시를 해주고,

![](../images/2020-11-17-bioconductor/Untitled77.png)

```r
class(corresponding_gene_name) # "character"
corresponding_gene_name[idx.queryHits]
```

해당하는 유전자들의 이름을 뽑는다. character 타입의 데이터는 인덱싱 연산, 즉 숫자로 데이터를 뽑을 수 있으며, TRUE, FALSE를 모든 인자에 주는 식으로 데이터를 뽑을 수도 있다.코드를 줄이면 다음과 같다.

![](../images/2020-11-17-bioconductor/Untitled78.png)

```r
corresponding_gene_name <- fdata.gr(달러표시)gene_name[subjectHits(overlaps)][queryHits((overlaps))==1]
length(corresponding_gene_name)
corresponding_gene_name
```

![](../images/2020-11-17-bioconductor/Untitled79.png)

# References

- Amezquita et al., Nature methods, 2019.12
- [https://en.wikipedia.org/wiki/Bioconductor](https://en.wikipedia.org/wiki/Bioconductor)
- Huber et al., 2015, Nat Methods
- [https://kkokkilkon.tistory.com/67](https://kkokkilkon.tistory.com/67)
- [https://sosal.kr/848](https://sosal.kr/848)